In [22]:
import numpy as np
%matplotlib qt
import matplotlib.pyplot as pl
import time
import pickle
import os

In [52]:
class State:
    def __init__(self,L:int,p:float,q:int,initial_state) -> None:
        assert L%2 == 0
        self.p = p
        self.q = q
        self.L = L
        # index = np.arange(0,2**L,1)
        self.data = initial_state.copy()
        self.log_Z = []
        
        self.data = np.zeros((2,)*L)
        self.data = initial_state
        
     

In [24]:
def eventransfer(State,a_structure):
    L = State.L
    os = State.data.reshape((4,)*(L//2))
    for x in range(L//2):
        if a_structure[x] == -1:
            U = State.U_t
        elif a_structure[x] == 1:
            U = State.U_k
        elif a_structure[x] == 0:
            U = State.U_haar
        os = np.tensordot(U,os,axes=(-1,x))
        os = np.moveaxis(os,0,x)
    os = np.reshape(os,(2,)*L)
    State.data = os


def oddtransfer(State,a_structure):
    L = State.L
    os = State.data
    os = np.moveaxis(os,0,-1) #bringing 1st spin to the last position
    os = np.reshape(os,(4,)*(L//2))

    for x in range(L//2):
        if a_structure[x] == -1:
            U = State.U_t
        elif a_structure[x] == 1:
            U = State.U_k
        elif a_structure[x] == 0:
            U = State.U_haar
        os = np.tensordot(U,os,axes=(-1,x))
        os = np.moveaxis(os,0,x)
    os = np.reshape(os,(2,)*L)
    os = np.moveaxis(os,-1,0)
    State.data = os

In [25]:
def no_of_down_spins(N):
    temp = np.arange(0,2**N,1,dtype=int)
    no_of_ones = np.zeros(2**N,dtype=float)
    for i in range(N):
        no_of_ones += temp%2
        temp = (temp/2).astype(int)
    return N - no_of_ones

def TopLayer(state,no_of_downs):
    """
    This function contracts the evolved state with the top layer.
    """

    ## Caluclate no. of down spins in each configuration
    # no_of_downs = no_of_down_spins(state.L)
    fac = np.sum(state.data.reshape(2**state.L)*((state.q)**(no_of_downs-(state.L)//2)))
    
    return np.log(fac)

In [26]:
def get_U_haar(q):
    U = np.zeros((4,4))
    U[0,0] = 1
    U[3,0] = 0
    U[3,3] = 1
    U[0,3] = 0
    U[3,1] = q/(q**2+1)
    U[3,2] = q/(q**2+1)
    U[0,1] = q/(q**2+1)
    U[0,2] = q/(q**2+1)
    return U

def get_U_t(p,q):
    U = np.zeros((4,4))
    U[0,0] = p**2
    U[3,0] = (1-p**2)/q**2
    U[3,3] = 1
    U[0,3] = 0
    U[3,1] = q*p**2/(q**2+1) + (1-p**2)/q
    U[3,2] = q*p**2/(q**2+1) + (1-p**2)/q
    U[0,1] = p**2*q/(q**2+1)
    U[0,2] = p**2*q/(q**2+1)
    return U

def get_U_k(p,q):
    U = np.zeros((4,4))
    U[0,0] = 1
    U[3,0] = 0
    U[3,3] = p**2
    U[0,3] = (1-p**2)/q**2
    U[0,2] = q*p**2/(q**2+1) + (1-p**2)/q
    U[0,1] = q*p**2/(q**2+1) + (1-p**2)/q
    U[3,2] = p**2*q/(q**2+1)
    U[3,1] = p**2*q/(q**2+1)
    return U

In [27]:
def encoded_bit_state(L,depth,q,BC='up'): 
    """
    This function evolves a single LOCAL bell pair to global bit by evolving by Haar dynamics. BC specify whether the initial boundary condition of the bell pair is 'up' or 'down' corresponding to Identity or Swap permutation respectively.
    """
    # up and down correspond to bottom boundary condition for the bell pair.
    
    initial_state = np.zeros((2,)*L)
    if BC == 'up':
        initial_state[1,:] = 1
    elif BC == 'down':
        initial_state[0,:] = 1
    else:
        print("BC parameter is wrong. It can only take 'up' or 'down' value")
    
    state = State(L=L,p=None,q=q,initial_state=initial_state)
    log_Z = []
     # Scrambling
    state.U_haar = get_U_haar(q)
    for t in range(depth):
        start = time.time()
        if t%2 == 0:
            eventransfer(state,[0]*(L//2))
        else:
            oddtransfer(state,[0]*(L//2))
        sd = np.sum(state.data)
        log_Z.append(np.log(sd))
        state.data = state.data/sd
    state.log_Z.append(np.sum(log_Z)) # storing the Z (partition function) for the encoding process

    return state


def load_bit_state(L,depth,q,BC='up'):
    filedir = 'data/encoded_bell_pairs'
    if not os.path.isdir(filedir):
        os.makedirs(filedir)
    
    filename = filedir + '/L='+str(L)+'_T='+str(depth)+'_q='+str(q)+'_'+BC
    if os.path.isfile(filename):
        with open(filename,'rb') as f:
            state = pickle.load(f)
        return state
    
    state = encoded_bit_state(L,depth,q,BC)
    with open(filename,'wb') as f:
        pickle.dump(state,f)
    
    return state
    

In [28]:
def get_U_array(U_t,U_k,U_haar,a_structure):
    """
    -1 corresponds to ancilla given to Environment
    +1 corresponds to ancilla given to Observer
    0 corresponds to there being no ancilla; that is the transfer matrix is that of Haar dynamics
    """
    
    U_array = np.zeros(a_structure.shape,dtype=object)
    U_array[np.where(a_structure==-1)] = U_t
    U_array[np.where(a_structure==1)] = U_k
    U_array[np.where(a_structure==0)] = U_haar
    # N = len(a_structure)
    # if a_structure == [-1]*N:
    #     U_array = [U_t]
    # elif a_structure == [1]*N:
    #     U_array = [U_k]
    # elif a_structure == [0]*N:
    #     U_array = [U_haar]

    


    # else:
    #     for x in range(N):
    #         if a_structure[x] == -1:
    #             U_array.append(U_t)
    #         elif a_structure[x] == 'k':
    #             U_array.append(U_k)
    #         elif a_structure[x] == 'h':
    #             U_array.append(U_haar)
    
    return U_array

In [29]:
## function to get free energy when the strength of coupling between S and ancillas is same everywhere
def free_energy_uniform(state: State, depth,ancilla_array : np.ndarray):
    
    """
    ancilla_string is a list whose elements are string of length L//2. For the element 't','k','h', the transfer matrix corresponding to having the ancilla traced out, kept in system, no ancilla respectively are applied. 
    """
    p = state.p
    q = state.q
    L = state.L
    assert np.shape(ancilla_array) == (depth,L)
    
    top_layer_factor = []
    ##
    state.U_t = get_U_t(p,q)
    state.U_k = get_U_k(p,q)
    state.U_haar = get_U_haar(q)

    no_of_downs = no_of_down_spins(L)

    for t in range(depth):
        # start = time.time()
        
        if t%2 == 0:
            eventransfer(state,ancilla_array[t])
        else:
            oddtransfer(state,ancilla_array[t])
        sd = np.sum(state.data)
        state.log_Z.append(np.log(sd))
        state.data = state.data/sd
        # print(time.time()-start)
        top_layer_factor.append(TopLayer(state,no_of_downs))
    

    return state,top_layer_factor


In [47]:
def get_ancilla_array(L,T,pattern='random',preference='E',seed=1): #preference decides to give preference to E or O for initial time step in case they cannot be equally distributed
    """
    -1 corresponds to ancilla given to Environment
    +1 corresponds to ancilla given to Observer
    0 corresponds to there being no ancilla; that is the transfer matrix is that of Haar dynamics
    """
    if pattern == 'checkerboard':
        a_structure = (np.indices((T,L)).sum(axis=0) % 2)
        if preference == 'E':
            a_structure = 2*a_structure - 1
        elif preference == 'O':
            a_structure = 1-2*a_structure

    if pattern == 'alternate':
        a_structure = np.ones((T,L))
        for t in range(T):
            if preference == 'E':
                if t%2==0:
                    a_structure[t,:] = -1
            elif preference == 'O':
                if t%2==1:
                    a_structure[t,:] = -1
    if pattern == 'random':
        rng = np.random.default_rng(seed=seed)
        a_structure = rng.integers(0,2,(T,L))*2-1
        
    return a_structure

In [48]:
## Calculate coherent QI for a single encoded bell pair.
F = {}
F_t = {}
for L in [8,10,12,14,16,18][:]:
    p_list = np.round(np.linspace(0,1,20),2)
    T = 2*L
    q = 2
    F[L] = []
    F_t[L] = {}
    for p in p_list:
        F_t[L][p] = []
        state_up = load_bit_state(L,2*L,q,'up')
        state_down = load_bit_state(L,2*L,q,'down')
        state_up.p = p
        state_down.p = p
        a_structure = get_ancilla_array(L,T,pattern='random',seed=1)
        state_up,top_layer_up= free_energy_uniform(state_up,T,a_structure)
        state_down, top_layer_down= free_energy_uniform(state_down,T,a_structure)

        for t in range(T):
            temp_up = np.sum(state_up.log_Z[:1+t+1]) + top_layer_up[t]
            temp_down = np.sum(state_down.log_Z[:1+t+1]) + top_layer_down[t]
            F_t[L][p].append(temp_down-temp_up)
        F[L].append(F_t[L][p][-1])
            

In [49]:
for L in [16]:
    for p in F_t[L]:
        T_data = np.arange(1,len(F_t[L][p])+1,1)[::2]/L
        pl.plot(T_data,np.array(F_t[L][p])[::2]/(np.log(q)),'-o',label = r'$p=$'+str(p))
pl.legend()

In [51]:
p_c = 0
nu = np.inf
for L in F:
    pl.plot((np.array(p_list)-p_c)*L**(1/nu),[np.array(F_t[L][p][-1])/np.log(q) for p in p_list],'-o',label = r'$L=$'+str(L))

pl.ylabel(r'$\Delta F$',fontsize=16)
pl.xlabel(r'$p$',fontsize=16)

pl.title(r'Haar random dynamics for single Bell pair,'+'\n'+ '$q=2,T=10L$, $p_O=p_E=0.5$',fontsize=16)

pl.legend(fontsize=16)
pl.tight_layout()

In [ ]:
p_list

array([0.05, 0.1 , 0.15, 0.2 , 0.25, 0.3 , 0.35, 0.4 , 0.45, 0.5 , 0.55,
       0.6 , 0.65, 0.7 , 0.75, 0.8 , 0.85, 0.9 , 0.95, 1.  ])